# Reshaping in Pandas with stack() and unstack()

This is a Notebook for the medium article [Reshaping a DataFrame with Pandas stack() and unstack()](https://bindichen.medium.com/reshaping-a-dataframe-with-pandas-stack-and-unstack-925dc9ce1289)

Please check out article for instructions

**License**: [BSD 2-Clause](https://opensource.org/licenses/BSD-2-Clause)


#### Version of packages used in this Notebook

In [1]:
import numpy as np
import pandas as pd

# Make sure your package version >= them
print('numpy: ', np.__version__)
print('pandas: ', pd.__version__)

numpy:  1.18.1
pandas:  1.1.4


# Tutorial

In [2]:
import pandas as pd
import numpy as np

## 1. Single level

In [16]:
df_single_level = pd.DataFrame(
    [['Mostly cloudy', 10], ['Sunny', 12]],
    index=['London', 'Oxford'],
    columns=['Weather', 'Wind']
)

df_single_level

,Weather,Wind
London,Mostly cloudy,10
Oxford,Sunny,12


In [17]:
df_single_level.stack()

London  Weather    Mostly cloudy
        Wind                  10
Oxford  Weather            Sunny
        Wind                  12
dtype: object

## 2. Multi level columns: simple case

In [24]:
multi_col_1 = pd.MultiIndex.from_tuples(
    [('Wind', 'mph'), ('Wind', 'm/s')]
)

In [25]:
df_multi_level_1 = pd.DataFrame(
    [[13, 5.5], [19, 8.5]],
    index=['London', 'Oxford'],
    columns=multi_col_1
)

df_multi_level_1

Wind     
        mph  m/s
London   13  5.5
Oxford   19  8.5

In [26]:
df_multi_level_1.stack()

Wind
London m/s   5.5
       mph  13.0
Oxford m/s   8.5
       mph  19.0

## 3. Multiple levels: Missing values

In [27]:
multi_col_2 = pd.MultiIndex.from_tuples(
    [('Wind', 'mph'), ('Temperature', '°C')]
)

In [30]:
df_multi_level_2 = pd.DataFrame(
    [[13, 8], [19, 6]],
    index=['London', 'Oxford'],
    columns=multi_col_2
)
df_multi_level_2

,Wind,Temperature
,mph,°C
London,13,8
Oxford,19,6


In [32]:
df_multi_level_2.stack()

Temperature  Wind
London mph          NaN  13.0
       °C           8.0   NaN
Oxford mph          NaN  19.0
       °C           6.0   NaN

## 4. Multiple levels: Prescribing the level(s) to be stacked

In [53]:
multi_col_2.get_level_values(0)

Index(['Wind', 'Temperature'], dtype='object')

In [54]:
multi_col_2.get_level_values(1)

Index(['mph', '°C'], dtype='object')

In [33]:
df_multi_level_2.stack(0)

mph   °C
London Temperature   NaN  8.0
       Wind         13.0  NaN
Oxford Temperature   NaN  6.0
       Wind         19.0  NaN

In [34]:
df_multi_level_2.stack([0, 1])

London  Temperature  °C      8.0
        Wind         mph    13.0
Oxford  Temperature  °C      6.0
        Wind         mph    19.0
dtype: float64

In [35]:
df_multi_level_2.stack([1, 0])

London  mph  Wind           13.0
        °C   Temperature     8.0
Oxford  mph  Wind           19.0
        °C   Temperature     6.0
dtype: float64

## 5. Multiple levels: Dropping missing values

In [40]:
df_multi_level_3 = pd.DataFrame(
    [[None, 10], [11, 7.0]],
    index=['London', 'Oxford'],
    columns=multi_col_2
)

In [41]:
df_multi_level_3

,Wind,Temperature
,mph,°C
London,NaN,10.0
Oxford,11.0,7.0


In [42]:
df_multi_level_3.stack(dropna=False)

Temperature  Wind
London mph          NaN   NaN
       °C          10.0   NaN
Oxford mph          NaN  11.0
       °C           7.0   NaN

In [44]:
df_multi_level_3.stack(dropna=True)

Temperature  Wind
London °C          10.0   NaN
Oxford mph          NaN  11.0
       °C           7.0   NaN

## 6. unstack

In [56]:
index = pd.MultiIndex.from_tuples([
  ('Oxford', 'Temperature'), 
  ('Oxford', 'Wind'),
  ('London', 'Temperature'), 
  ('London', 'Wind')
])

In [65]:
index

MultiIndex([('Oxford', 'Temperature'),
            ('Oxford',        'Wind'),
            ('London', 'Temperature'),
            ('London',        'Wind')],
           )

In [61]:
s = pd.Series([1,2,3,4], index=index)
s

Oxford  Temperature    1
        Wind           2
London  Temperature    3
        Wind           4
dtype: int64

In [62]:
s.unstack()

,Temperature,Wind
London,3,4
Oxford,1,2


In [63]:
# The equiv
s.unstack(level=-1)

,Temperature,Wind
London,3,4
Oxford,1,2


In [64]:
s.unstack(level=0)

,London,Oxford
Temperature,3,1
Wind,4,2


## 7. unstack: more levels

In [86]:
index = pd.MultiIndex.from_tuples([
  ('Oxford', 'Weather', '01-01-2022'), 
  ('Oxford', 'Temperature', '01-01-2022'), 
  ('Oxford', 'Weather', '02-01-2022'),
  ('Oxford', 'Temperature', '02-01-2022'),
  ('London', 'Weather', '01-01-2022'), 
  ('London', 'Temperature', '01-01-2022'),
  ('London', 'Weather', '02-01-2022'),
  ('London', 'Temperature', '02-01-2022'),
])

In [87]:
s = pd.Series(['Sunny', 10, 'Shower', 7, 'Shower', 5, 'Sunny', 8], index=index)
s

Oxford  Weather      01-01-2022     Sunny
        Temperature  01-01-2022        10
        Weather      02-01-2022    Shower
        Temperature  02-01-2022         7
London  Weather      01-01-2022    Shower
        Temperature  01-01-2022         5
        Weather      02-01-2022     Sunny
        Temperature  02-01-2022         8
dtype: object

In [89]:
s.unstack()

01-01-2022 02-01-2022
London Temperature          5          8
       Weather         Shower      Sunny
Oxford Temperature         10          7
       Weather          Sunny     Shower

In [90]:
s.unstack().unstack()

01-01-2022          02-01-2022        
       Temperature Weather Temperature Weather
London           5  Shower           8   Sunny
Oxford          10   Sunny           7  Shower

In [92]:
s.unstack(level=[2, 1])

01-01-2022             02-01-2022            
          Weather Temperature    Weather Temperature
London     Shower           5      Sunny           8
Oxford      Sunny          10     Shower           7

In [91]:
s.unstack().unstack().unstack()

01-01-2022  Temperature  London         5
                         Oxford        10
            Weather      London    Shower
                         Oxford     Sunny
02-01-2022  Temperature  London         8
                         Oxford         7
            Weather      London     Sunny
                         Oxford    Shower
dtype: object

In [93]:
s.unstack(level=[0,1])

Oxford              London            
           Weather Temperature Weather Temperature
01-01-2022   Sunny          10  Shower           5
02-01-2022  Shower           7   Sunny           8

### Thanks for reading 

This is a Notebook for the medium article [Reshaping a DataFrame with Pandas stack() and unstack()](https://bindichen.medium.com/reshaping-a-dataframe-with-pandas-stack-and-unstack-925dc9ce1289)

Please check out article for instructions

**License**: [BSD 2-Clause](https://opensource.org/licenses/BSD-2-Clause)